In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
train.head()

,index,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,native-country,Y
0,0,22,Private,132618,Some-college,12,Never-married,Tech-support,Own-child,White,Female,United-States,0
1,1,22,Private,132655,Bachelors,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,Mexico,1
2,2,23,?,132674,HS-grad,8,Never-married,Other-service,Not-in-family,White,Female,United-States,0
3,3,36,Private,132642,Bachelors,13,Married-civ-spouse,Sales,Husband,White,Male,United-States,1
4,4,49,Private,132646,Assoc-voc,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,United-States,1


In [13]:
len(train.columns)

13

## column
'workclass', 'education','marital-status', 'occupation', 'relationship', 'race', 'sex','native-country'

In [5]:
train_x=train.drop(['Y'], axis=1)
train_y=train['Y']
test_x=test.copy()

cat_cols= ['workclass', 'education','marital-status', 'occupation', 'relationship', 'race', 'sex','native-country']

train_x = train_x.drop(['fnlwgt'],axis=1)
train_x = train_x.drop(['race'],axis=1)

cat_cols= ['workclass', 'education','marital-status', 'occupation', 'relationship', 'sex','native-country']

In [6]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_profiling.profile_report

# drop_col
'fnlwgt',''race', 'df_index'

In [7]:
# labelencoder

# ラベルエンコーディング（OrdinalEncoder）
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

"""
'index', 'age', 'workclass', 'fnlwgt', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'native-country',
"""



train_x = train_x.dropna()
test_x = test_x.dropna()

train_x.head()



# -----------------------------------
# target encoding
# -----------------------------------
from sklearn.model_selection import KFold

# 変数をループしてtarget encoding
for c in cat_cols:
    # 学習データ全体で各カテゴリにおけるtargetの平均を計算
    data_tmp = pd.DataFrame({c: train_x[c], 'target': train_y})
    target_mean = data_tmp.groupby(c)['target'].mean()
    # テストデータのカテゴリを置換
    test_x[c] = test_x[c].map(target_mean)

    # 学習データの変換後の値を格納する配列を準備
    tmp = np.repeat(np.nan, train_x.shape[0])

    # 学習データを分割
    kf = KFold(n_splits=4, shuffle=True, random_state=72)
    for idx_1, idx_2 in kf.split(train_x):
        # out-of-foldで各カテゴリにおける目的変数の平均を計算
        target_mean = data_tmp.iloc[idx_1].groupby(c)['target'].mean()
        # 変換後の値を一時配列に格納
        tmp[idx_2] = train_x[c].iloc[idx_2].map(target_mean)

    # 変換後のデータで元の変数を置換
    train_x[c] = tmp



In [8]:
import lightgbm as lgb

from sklearn import metrics
from sklearn.model_selection import StratifiedKFold


def get_evaluate(y_test, predict):

    fpr, tpr, thr_arr = metrics.roc_curve(y_test, predict)

    auc = metrics.auc(fpr, tpr)
    precision = metrics.precision_score(y_test, predict)
    recall = metrics.recall_score(y_test, predict)      

    return auc, precision, recall

from sklearn.model_selection import StratifiedKFold
# kfoldの分割数
k = 10
skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=0)

lgbm_params = {'objective': 'binary'}

auc_list = []
precision_list = []
recall_list = []

X = train_x
y = train_y

# kfoldで分割
for train_index, test_index in skf.split(X, y):

    X_train = X.iloc[train_index]
    y_train = y.iloc[train_index]
    X_test = X.iloc[test_index]
    y_test = y.iloc[test_index]

    # データセットを生成する
    lgb_train = lgb.Dataset(X_train, y_train)

    # 学習
    model = lgb.train(lgbm_params, lgb_train)

    predict_proba = model.predict(X_test, num_iteration=model.best_iteration)
    # predict_probaが0.5以上なら1とする
    predict = [0 if i < 0.5 else 1 for i in predict_proba]

    auc, precision, recall = get_evaluate(y_test, predict)

    print('AUC:{}, precision:{}, recall:{}'.format(auc, precision, recall))

    auc_list.append(auc)
    precision_list.append(precision)
    recall_list.append(recall)

#kfoldの平均値を取得
print('Kfold平均 AUC:{}, precision:{}, recall:{}'.format(np.mean(auc_list), 
                                                         np.mean(precision_list), 
                                                         np.mean(recall_list)))
                                                         

[LightGBM] [Info] Number of positive: 2223, number of negative: 6957
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000106 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 520
[LightGBM] [Info] Number of data points in the train set: 9180, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.242157 -> initscore=-1.140891
[LightGBM] [Info] Start training from score -1.140891
AUC:0.8421524006054544, precision:0.7872340425531915, recall:0.7489878542510121
[LightGBM] [Info] Number of positive: 2223, number of negative: 6957
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000061 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 521
[LightGBM] [Info] Number of data

In [11]:
import numpy as np
import pandas as pd

from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import catboost as cb
from catboost import CatBoost
from catboost import Pool




In [12]:
# データセットを生成する
train_pool = Pool(train_x, train_y)
test_pool = Pool(X_test, y_test)

# LightGBMのハイパーパラメータを設定する
params = {'loss_function': 'AUCE',
         'num_boost_round': 100,}

# 上記のパラメータでモデルを学習する
model = CatBoost(params)
model.fit(train_pool)

y_pred = model.predict(X_test)

# RMSE を計算する
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print('RMSE:',round(rmse,3))
"""output
RMSE: 2.947
"""

# 特徴量の重要度を取得する
feature_importance = model.get_feature_importance()
print(feature_importance)


CatBoostError: C:/Program Files (x86)/Go Agent/pipelines/BuildMaster/catboost.git/tools/enum_parser/enum_serialization_runtime/enum_runtime.cpp:32: Key 'AUCE' not found in enum ELossFunction. Valid options are: 'Logloss', 'CrossEntropy', 'CtrFactor', 'RMSE', 'Lq', 'MAE', 'Quantile', 'Expectile', 'LogLinQuantile', 'MAPE', 'Poisson', 'MSLE', 'MedianAbsoluteError', 'SMAPE', 'Huber', 'Tweedie', 'Cox', 'RMSEWithUncertainty', 'MultiClass', 'MultiClassOneVsAll', 'PairLogit', 'PairLogitPairwise', 'YetiRank', 'YetiRankPairwise', 'QueryRMSE', 'QuerySoftMax', 'QueryCrossEntropy', 'StochasticFilter', 'LambdaMart', 'StochasticRank', 'PythonUserDefinedPerObject', 'PythonUserDefinedMultiRegression', 'UserPerObjMetric', 'UserQuerywiseMetric', 'R2', 'NumErrors', 'FairLoss', 'AUC', 'Accuracy', 'BalancedAccuracy', 'BalancedErrorRate', 'BrierScore', 'Precision', 'Recall', 'F1', 'TotalF1', 'MCC', 'ZeroOneLoss', 'HammingLoss', 'HingeLoss', 'Kappa', 'WKappa', 'LogLikelihoodOfPrediction', 'NormalizedGini', 'P

In [ ]:
train_x

In [ ]:
from catboost import Pool
import numpy as np

# カテゴリのカラムのみを抽出
categorical_features_indices = np.where(train_x.dtypes != np.float)[0]

# データセットの作成。Poolで説明変数、目的変数、
# カラムのデータ型を指定できる
train_pool = Pool(train_x, train_y, cat_features=categorical_features_indices)
validate_pool = Pool(test_x, y_test, cat_features=categorical_features_indices)

# 分類用のインスタンスを作成
from catboost import CatBoostClassifier
model = CatBoostClassifier(custom_loss=['Accuracy'],
                           random_seed=42)

# CatBoost, CatBoostRegressorも存在するが損失関数が異なるだけ
# CatBoostClassifierの損失関数はlog_lossデフォルト

model.fit(train_pool, 
          eval_set=validate_pool,    # 検証用データ
          early_stopping_rounds=10,  # 10回以上精度が改善しなければ中止
          use_best_model=True,       # 最も精度が高かったモデルを使用するかの設定
          plot=True)                 # 誤差の推移を描画するか否かの設定

In [ ]:

#標準化
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(train_x[num_cols])

train_x[num_cols] = scaler.transform(train_x[num_cols])
test_x[num_cols] = scaler.transform(test_x[num_cols])

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.preprocessing import PolynomialFeatures

# 前処理
df = train
df["AG_ratio"].fillna(df["Alb"] / (df["TP"] - df["Alb"]), inplace=True)
df.drop_duplicates(inplace=True)
df.reset_index(drop=True, inplace=True)
X = df.drop(["disease"], axis=1)
y = df["disease"]

# Gender列を除外（数量変数のデータに絞る）
X_target = X.drop(["Gender"], axis=1)

# 多項式・交互作用特徴量の生成
polynomial = PolynomialFeatures(degree=2, include_bias=False)
polynomial_arr = polynomial.fit_transform(X_target)

# polynomial_arrのデータフレーム化 （※カラムはshape[1]でpolynomial_arrの列数分だけ出力）
X_polynomial = pd.DataFrame(polynomial_arr, columns=["poly" + str(x) for x in range(polynomial_arr.shape[1])])


# 生成した多項式・交互作用特徴量の表示
print(X_polynomial.shape)
print(X_polynomial.head())


In [ ]:

#診断ガイドラインを参照して差分の特徴量を作成する

#AST_GOT
train_x.loc[train_x['AST_GOT'] > 38, 'AST_GOT'] = train_x['AST_GOT'] - 38
train_x.loc[~(train_x['AST_GOT'] > 38), 'AST_GOT'] = 0

test_x.loc[train_x['AST_GOT'] > 38, 'AST_GOT'] = test_x['AST_GOT'] - 38
test_x.loc[~(train_x['AST_GOT'] > 38), 'AST_GOT'] = 0

#ALT_GPT
train_x.loc[train_x['ALT_GPT'] > 44, 'ALT_GPT'] = train_x['ALT_GPT'] - 44
train_x.loc[~(train_x['ALT_GPT'] > 44), 'ALT_GPT'] = 0

test_x.loc[train_x['ALT_GPT'] > 38, 'ALT_GPT'] = test_x['ALT_GPT'] - 44
test_x.loc[~(train_x['ALT_GPT'] > 38), 'ALT_GPT'] = 0

#AG_ratio
train_x.loc[train_x['AG_ratio'] > 1.8, 'AG_ratio'] = train_x['AG_ratio'] - 1.8
train_x.loc[~(train_x['AG_ratio'] > 1.8), 'AG_ratio'] = 0

test_x.loc[train_x['AG_ratio'] > 1.8, 'AG_ratio'] = test_x['AG_ratio'] - 1.8
test_x.loc[~(train_x['AG_ratio'] > 1.8), 'AG_ratio'] = 0


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel

# 組み込み法に使うモデルの指定
fs_model = LogisticRegression(penalty='l1', random_state=0)

# 閾値の指定
fs_threshold = "mean"

# 組み込み法モデルの初期化
selector = SelectFromModel(fs_model, threshold=fs_threshold)


# ライブラリのimport
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.feature_selection import SelectFromModel

# 組み込み法のモデル、閾値の指定
fs_model = LogisticRegression(penalty='l2', random_state=0)
fs_threshold = "mean"
# 組み込み法モデルの初期化
selector = SelectFromModel(fs_model, threshold=fs_threshold)

# 特徴量選択の実行
selector.fit(X_polynomial, y)
mask = selector.get_support()

# 選択された特徴量だけのサンプル取得
X_polynomial_masked = X_polynomial.loc[:, mask]

print("選択された特徴量の表示（最初の5行）")
print(X_polynomial_masked.head())
print("選択された特徴量の数の確認")
print(X_polynomial_masked.shape)



In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.preprocessing import PolynomialFeatures
from sklearn.feature_selection import SelectFromModel

# 学習用・評価用データの分割（元の説明変数Xの代わりに、特徴量選択後のX_polynomial_maskedを使う）
X_train, X_test, y_train, y_test = train_test_split(X_polynomial_masked, y, test_size=0.3, random_state=0)
train_x, test_x, train_x, test_y = train_test_split(X_polynomial_masked, y, test_size=0.3, random_state=0)
# モデルの学習・予測
model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict_proba(X_test)[:, 1]


In [ ]:
#light gbm
"""
from optuna.integration import lightgbm as lgb
ts = time.time()

dtrain = lgb.Dataset(x_train, label=y_train)
eval_data = lgb.Dataset(x_val, label=y_val)

param = {
        'objective': 'regression',
        'metric': 'rmse',
        'verbosity': -1,
        'boosting_type': 'gbdt',
    }

best = lgb.train(param, 
                 dtrain,
                 valid_sets=eval_data,
                 early_stopping_rounds=100)

time.time() - ts

"""
import lightgbm as lgb

from sklearn import metrics
from sklearn.model_selection import StratifiedKFold


def get_evaluate(y_test, predict):

    fpr, tpr, thr_arr = metrics.roc_curve(y_test, predict)

    auc = metrics.auc(fpr, tpr)
    precision = metrics.precision_score(y_test, predict)
    recall = metrics.recall_score(y_test, predict)      

    return auc, precision, recall

from sklearn.model_selection import StratifiedKFold
# kfoldの分割数
k = 2
skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=0)

lgbm_params = {'objective': 'binary'}

auc_list = []
precision_list = []
recall_list = []

X = train_x
y = train_y

# kfoldで分割
for train_index, test_index in skf.split(X, y):

    X_train = X.iloc[train_index]
    y_train = y.iloc[train_index]
    X_test = X.iloc[test_index]
    y_test = y.iloc[test_index]

    # データセットを生成する
    lgb_train = lgb.Dataset(X_train, y_train)

    # 学習
    model = lgb.train(lgbm_params, lgb_train)

    predict_proba = model.predict(X_test, num_iteration=model.best_iteration)
    # predict_probaが0.5以上なら1とする
    predict = [0 if i < 0.5 else 1 for i in predict_proba]

    auc, precision, recall = get_evaluate(y_test, predict)

    print('AUC:{}, precision:{}, recall:{}'.format(auc, precision, recall))

    auc_list.append(auc)
    precision_list.append(precision)
    recall_list.append(recall)

#kfoldの平均値を取得
print('Kfold平均 AUC:{}, precision:{}, recall:{}'.format(np.mean(auc_list), 
                                                         np.mean(precision_list), 
                                                         np.mean(recall_list)))
                                                         

In [ ]:
"""
# ROC曲線の描画（偽陽性率、真陽性率、閾値の算出）
fpr, tpr, thresholds = roc_curve(y_true=test_y, y_score=pred_y)
plt.plot(fpr, tpr, label='roc curve')
plt.plot([0, 1], [0, 1], linestyle=':', label='random')
plt.plot([0, 0, 1], [0, 1, 1], linestyle=':', label='ideal')
plt.legend()
plt.xlabel('false positive rate')
plt.ylabel('true positive rate')
plt.show()

# AUCスコアの算出
auc_score = roc_auc_score(y_true=y_test, y_score=y_pred)
print("AUC:", auc_score)

"""

In [ ]:
"""
#catboost

from catboost import Pool
import numpy as np

# カテゴリのカラムのみを抽出
categorical_features_indices = np.where(X.dtypes != np.float)[0]

# データセットの作成。Poolで説明変数、目的変数、
# カラムのデータ型を指定できる
train_x = Pool(X_train, y_train, cat_features=categorical_features_indices)
validate_pool = Pool(X_test, y_test, cat_features=categorical_features_indices)

"""

In [ ]:
pred = model.predict(test_x)
pred_label = np.where(pred>0.5,1,0)

submission = pd.DataFrame()
submission.to_csv('sub.csv')